# Count the occurence of the dictionary words in Yelp Reviews and Tips data

In [1]:
import pandas as pd
import numpy as np
import re
import json
import csv

## Read Yelp Review and Tips data and mark the occurence of dictionary words

In [11]:
iy = pd.read_csv('../dataset/iy_ready2.csv', dtype={'ID': np.int64})

In [12]:
review = pd.read_csv('../dataset/ac_all_review.csv', encoding='latin-1')
review.shape # 199434 reviews

(199434, 10)

In [13]:
tips = pd.read_csv('../dataset/ac_all_tips.csv', encoding='latin-1')
tips.shape # 35143 tips

(35143, 6)

In [14]:
# Dictionary

# Unhygenic
unh=["gross", "mess", "sticky", "smell", "restroom", "dirty", "awful", "bad", "horrible", "worst"]
# Basic Ingredients
ing=["beef", "pork", "noodle", "egg", "soy","ramen", "pho"]
# Cuisines
# cui=["vietnamese", "dim sum", "thai", "mexican","japanese", "chinese", "american", "pizza", "sushi", "indian","italian", "asian"]
# Sentiment
sen=["cheap", "never", "yell"]
# Service & Atmosphere
sna= ["cash", "worth", "district", "delivery","think", "really", "thing", "parking", "always", "usually", "definitely",
    "door", "wait", "sticker","shock", "student","size"," dry", "too salty", "inedible" , "pool"]
unhygenic = [unh, ing, sen, sna]

# Hygenic
hhy = ["clean"]
# Cooking Method & Garnish: 
hck = ["brew", "frosting", "grill","crush", "crust", "taco", "burrito", "toast"]
#Healthy or Fancier Ingredients: 
hing = ["celery", "calamity","wine", "broccoli", "salad", "flatbread", "olive", "pesto"]
# Cuisines: 
# hcui = ["breakfast", "fish & chips", "fast food", "german", "diner", "belgian", "european", "sandwiches","vegetarian"] 
#Whom & When: 
hsit = ["date", "weekend", "our", "husband","evening", "night"]
#Sentiment: 
hsent = ["love", "yummy", "generous", "friendly", "great","nice", "excellent", "awesome", "delicious", "amazing", "best"]
#Service & Atmosphere: 
hsvat = ["selection", "attitude","atmosphere", "ambiance", "pretentious"]
hygenic = [hhy, hck, hing, hsit, hsent, hsvat]

In [15]:
for i in tips.iterrows():
    if np.isin(i[1]['business_id'], list(iy['business_id'].unique())):
        tips.loc[i[0], 'food_facility'] = 1
        toks = re.findall("[\w']+|[.,!?;]", tips.iloc[i[0]]['text'].lower())
        
        for idx, dic in enumerate(unhygenic):
            if np.isin(toks, dic).any():
                tips.loc[i[0], "unh_"+str(idx)] = 1
        for idx, dic in enumerate(hygenic):
            if np.isin(toks, dic).any():
                tips.loc[i[0], "hyg_"+str(idx)] = 1
        
    else:
        tips.loc[i[0], 'food_facility'] = 0 # if it is 0, it doesn't belong to 'food facility'

In [16]:
for i in review.iterrows():
    if np.isin(i[1]['business_id'], list(iy['business_id'].unique())):
        review.loc[i[0], 'food_facility'] = 1
        
        toks = re.findall("[\w']+|[.,!?;]", review.iloc[i[0]]['text'].lower())
        for idx, dic in enumerate(unhygenic):
            if np.isin(toks, dic).any():
                review.loc[i[0], "unh_"+str(idx)] = 1
        for idx, dic in enumerate(hygenic):
            if np.isin(toks, dic).any():
                review.loc[i[0], "hyg_"+str(idx)] = 1
    else:
        review.loc[i[0], 'food_facility'] = 0 # if it is 0, it doesn't belong to 'food facility'

In [17]:
tips['food_facility'].value_counts()

0.0    19198
1.0    15945
Name: food_facility, dtype: int64

In [18]:
review['food_facility'].value_counts()

0.0    104508
1.0     94926
Name: food_facility, dtype: int64

In [19]:
tips = tips[tips['food_facility']==1]
review = review[review['food_facility']==1]

In [21]:
tips.columns.values

array(['Unnamed: 0', 'business_id', 'date', 'likes', 'text', 'user_id',
       'food_facility', 'hyg_4', 'hyg_3', 'hyg_2', 'unh_3', 'unh_1',
       'hyg_5', 'hyg_1', 'unh_0', 'unh_2', 'hyg_0'], dtype=object)

In [22]:
review.columns.values

array(['Unnamed: 0', 'business_id', 'cool', 'date', 'funny', 'review_id',
       'stars', 'text', 'useful', 'user_id', 'food_facility', 'unh_3',
       'hyg_4', 'hyg_3', 'unh_0', 'unh_2', 'hyg_1', 'hyg_5', 'hyg_2',
       'unh_1', 'hyg_0'], dtype=object)

In [25]:
# Fill na with 0
# review.iloc[:,15:]
review.iloc[:, 11:] = review.iloc[:, 11:].fillna(value=0)
tips.iloc[:, 7:] = tips.iloc[:, 7:].fillna(value=0)

In [26]:
print(review.columns)
print(tips.columns)

Index(['Unnamed: 0', 'business_id', 'cool', 'date', 'funny', 'review_id',
       'stars', 'text', 'useful', 'user_id', 'food_facility', 'unh_3', 'hyg_4',
       'hyg_3', 'unh_0', 'unh_2', 'hyg_1', 'hyg_5', 'hyg_2', 'unh_1', 'hyg_0'],
      dtype='object')
Index(['Unnamed: 0', 'business_id', 'date', 'likes', 'text', 'user_id',
       'food_facility', 'hyg_4', 'hyg_3', 'hyg_2', 'unh_3', 'unh_1', 'hyg_5',
       'hyg_1', 'unh_0', 'unh_2', 'hyg_0'],
      dtype='object')


In [27]:
review = review.drop(['Unnamed: 0', 'cool', 'funny', 'review_id', 'text', 'useful', 'user_id', 'food_facility'], axis=1)
tips = tips.drop(['Unnamed: 0', 'likes', 'text', 'user_id', 'food_facility'], axis=1)

In [31]:
rt = review.append(tips)
rt.head()

,business_id,date,hyg_0,hyg_1,hyg_2,hyg_3,hyg_4,hyg_5,stars,unh_0,unh_1,unh_2,unh_3
200,3kPqtmL3EdfhCwxfIHT_iQ,2016-07-29,0.0,0.0,0.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,1.0
201,3kPqtmL3EdfhCwxfIHT_iQ,2016-11-12,0.0,0.0,0.0,0.0,1.0,0.0,5.0,0.0,0.0,0.0,0.0
202,3kPqtmL3EdfhCwxfIHT_iQ,2012-02-08,0.0,0.0,0.0,1.0,1.0,0.0,4.0,0.0,0.0,0.0,1.0
203,3kPqtmL3EdfhCwxfIHT_iQ,2015-01-04,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
204,3kPqtmL3EdfhCwxfIHT_iQ,2016-07-29,0.0,1.0,0.0,0.0,1.0,0.0,5.0,0.0,0.0,0.0,1.0


## Read a dataframe with Inspection data and Yelp main data combined

In [39]:
iy = pd.read_csv('../dataset/iy_ready2.csv', dtype={'ID': np.int64})

In [40]:
iy = iy.merge(rt, on='business_id')

In [41]:
print(iy.shape)
iy.head()

(617024, 54)


,Unnamed: 0,Encounter,ID,Facility Name,Bus St Date,Description,NUM,Street,City,ZIP,...,hyg_1,hyg_2,hyg_3,hyg_4,hyg_5,stars_y,unh_0,unh_1,unh_2,unh_3
0,0,201201030015,44322,Michael's Pizza Bar Restaurant,1989-01-31,Restaurant with Liquor,2612,Sarah St,Pittsburgh,15203,...,0.0,0.0,1.0,1.0,1.0,5.0,0.0,0.0,0.0,0.0
1,0,201201030015,44322,Michael's Pizza Bar Restaurant,1989-01-31,Restaurant with Liquor,2612,Sarah St,Pittsburgh,15203,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0,201201030015,44322,Michael's Pizza Bar Restaurant,1989-01-31,Restaurant with Liquor,2612,Sarah St,Pittsburgh,15203,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,1.0
3,0,201201030015,44322,Michael's Pizza Bar Restaurant,1989-01-31,Restaurant with Liquor,2612,Sarah St,Pittsburgh,15203,...,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0
4,0,201201030015,44322,Michael's Pizza Bar Restaurant,1989-01-31,Restaurant with Liquor,2612,Sarah St,Pittsburgh,15203,...,0.0,0.0,0.0,1.0,0.0,4.0,0.0,0.0,0.0,1.0


In [48]:
print("There has been approximately 10,000 restaurants in the original Yelp data, but now it has only {0} restaurants.".format(iy['ID'].unique().shape[0]))

There has been approximately 10,000 restaurants in the original Yelp data, but now it has only 1942 restaurants.


In [42]:
iy.columns

Index(['Unnamed: 0', 'Encounter', 'ID', 'Facility Name', 'Bus St Date',
       'Description', 'NUM', 'Street', 'City', 'ZIP', 'Inspect Dt',
       'Start Time', 'End Time', 'Low', 'Medium', 'High', 'business_id',
       'address', 'attributes', 'categories', 'city', 'name', 'neighborhood',
       'postal_code', 'review_count', 'stars_x', 'state', 'day_diff', 'price',
       'cuisine_Asian Fusion', 'cuisine_Vegetarian', 'cuisine_Chinese',
       'cuisine_American (Traditional)', 'cuisine_Italian', 'cuisine_Mexican',
       'cuisine_Thai', 'cuisine_others', 'cuisine_American (New)',
       'cuisine_Indian', 'cuisine_Japanese', 'cuisine_Greek',
       'cuisine_Mediterranean', 'date', 'hyg_0', 'hyg_1', 'hyg_2', 'hyg_3',
       'hyg_4', 'hyg_5', 'stars_y', 'unh_0', 'unh_1', 'unh_2', 'unh_3'],
      dtype='object')

In [43]:
iym = iy.drop(['Unnamed: 0', 'NUM', 'Street', 'City', 'ZIP', 'Start Time', 'End Time', 'attributes', 'categories', 'city',
              'name', 'neighborhood', 'postal_code', 'review_count', 'stars_x', 'state', 'day_diff'], axis=1)

In [44]:
iym.columns.values

array(['Encounter', 'ID', 'Facility Name', 'Bus St Date', 'Description',
       'Inspect Dt', 'Low', 'Medium', 'High', 'business_id', 'address',
       'price', 'cuisine_Asian Fusion', 'cuisine_Vegetarian',
       'cuisine_Chinese', 'cuisine_American (Traditional)',
       'cuisine_Italian', 'cuisine_Mexican', 'cuisine_Thai',
       'cuisine_others', 'cuisine_American (New)', 'cuisine_Indian',
       'cuisine_Japanese', 'cuisine_Greek', 'cuisine_Mediterranean',
       'date', 'hyg_0', 'hyg_1', 'hyg_2', 'hyg_3', 'hyg_4', 'hyg_5',
       'stars_y', 'unh_0', 'unh_1', 'unh_2', 'unh_3'], dtype=object)

In [49]:
# Change data type
iym['date'] = pd.to_datetime(iym['date'])
iym['Inspect Dt'] = pd.to_datetime(iym['Inspect Dt'])
iym['day_diff']=0

# Calculate day difference
def get_day_diff(row):
    return (row['Inspect Dt'] - row['date']).days

iym['day_diff'] = iym.apply(get_day_diff, axis=1)

In [57]:
# Use only past 1 year data
iy12 = iym[(iym['day_diff']<=365) & (iym['day_diff']>0)]
iy12['count']=1
print(iy12.shape)
iy12.head()

(104799, 39)


C:\Dev\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,Encounter,ID,Facility Name,Bus St Date,Description,Inspect Dt,Low,Medium,High,business_id,...,hyg_3,hyg_4,hyg_5,stars_y,unh_0,unh_1,unh_2,unh_3,day_diff,count
53,201201030015,44322,Michael's Pizza Bar Restaurant,1989-01-31,Restaurant with Liquor,2012-01-03,0.0,0.0,0.0,38aN0fB_zHtkw1kb3lSj8g,...,1.0,1.0,0.0,5.0,0.0,0.0,1.0,1.0,80,1
68,201201030015,44322,Michael's Pizza Bar Restaurant,1989-01-31,Restaurant with Liquor,2012-01-03,0.0,0.0,0.0,38aN0fB_zHtkw1kb3lSj8g,...,0.0,1.0,1.0,NaN,0.0,0.0,0.0,0.0,322,1
71,201201030015,44322,Michael's Pizza Bar Restaurant,1989-01-31,Restaurant with Liquor,2012-01-03,0.0,0.0,0.0,38aN0fB_zHtkw1kb3lSj8g,...,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,276,1
72,201201030015,44322,Michael's Pizza Bar Restaurant,1989-01-31,Restaurant with Liquor,2012-01-03,0.0,0.0,0.0,38aN0fB_zHtkw1kb3lSj8g,...,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,221,1
73,201201030015,44322,Michael's Pizza Bar Restaurant,1989-01-31,Restaurant with Liquor,2012-01-03,0.0,0.0,0.0,38aN0fB_zHtkw1kb3lSj8g,...,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,83,1


In [58]:
iy12.columns.values

array(['Encounter', 'ID', 'Facility Name', 'Bus St Date', 'Description',
       'Inspect Dt', 'Low', 'Medium', 'High', 'business_id', 'address',
       'price', 'cuisine_Asian Fusion', 'cuisine_Vegetarian',
       'cuisine_Chinese', 'cuisine_American (Traditional)',
       'cuisine_Italian', 'cuisine_Mexican', 'cuisine_Thai',
       'cuisine_others', 'cuisine_American (New)', 'cuisine_Indian',
       'cuisine_Japanese', 'cuisine_Greek', 'cuisine_Mediterranean',
       'date', 'hyg_0', 'hyg_1', 'hyg_2', 'hyg_3', 'hyg_4', 'hyg_5',
       'stars_y', 'unh_0', 'unh_1', 'unh_2', 'unh_3', 'day_diff', 'count'], dtype=object)

In [53]:
# Group by inspection
# Each row represents a unique inspection
iy12a = iy12.groupby(['Encounter', 'ID', 'Facility Name', 'Bus St Date', 'Description', 'Inspect Dt', 
                      'Low', 'Medium', 'High', 'business_id', 'address', 'price'],
                     as_index=False)['cuisine_Asian Fusion', 'cuisine_Vegetarian', 'cuisine_Chinese', 
                                     'cuisine_American (Traditional)', 'cuisine_Italian', 'cuisine_Mexican', 
                                     'cuisine_Thai', 'cuisine_others', 'cuisine_American (New)', 'cuisine_Indian', 
                                     'cuisine_Japanese', 'cuisine_Greek', 'cuisine_Mediterranean', 
                                     'hyg_0', 'hyg_1', 'hyg_2', 'hyg_3', 'hyg_4', 'hyg_5', 
                                     'unh_0', 'unh_1', 'unh_2', 'unh_3', 'stars_y'].mean()

In [59]:
iy12c = iy12.groupby(['Encounter', 'ID', 'Facility Name', 'Bus St Date', 'Description', 'Inspect Dt', 
                      'Low', 'Medium', 'High', 'business_id', 'address', 'price'],
                     as_index=False)['count'].sum()

In [60]:
iy12.shape

(104799, 39)

In [63]:
iy12th = pd.concat([iy12a, iy12c[['count']]], axis=1)
iy12th = iy12th[iy12th['count']>=5] # Minimum review or tips count to be included
iy12th.shape

(4572, 37)

In [64]:
iy12th.head()

,Encounter,ID,Facility Name,Bus St Date,Description,Inspect Dt,Low,Medium,High,business_id,...,hyg_2,hyg_3,hyg_4,hyg_5,unh_0,unh_1,unh_2,unh_3,stars_y,count
0,201201030015,44322,Michael's Pizza Bar Restaurant,1989-01-31,Restaurant with Liquor,2012-01-03,0.0,0.0,0.0,38aN0fB_zHtkw1kb3lSj8g,...,0.000000,0.200000,0.400000,0.200000,0.000000,0.000000,0.200000,0.200000,5.000000,5
1,201201040023,200707190005,China Palace,2007-07-11,Chain Restaurant with Liquor,2012-01-04,0.0,0.0,0.0,f2FfutZhb4F-m1Ob0EdYaw,...,0.090909,0.181818,0.727273,0.181818,0.181818,0.545455,0.363636,0.727273,3.700000,11
3,201201050018,1795,Robert Wholey Co,1992-12-16,Supermarket,2012-01-05,0.0,0.0,0.0,hsRjct8UiUHU54B983GmcA,...,0.064516,0.129032,0.612903,0.096774,0.129032,0.064516,0.129032,0.483871,4.095238,31
8,201201050040,200709250001,Miller's Seafood House,2007-09-24,Restaurant with Liquor,2012-01-05,0.0,0.0,0.0,jjullAo_5_q3x3yagyLmMA,...,0.000000,0.200000,0.600000,0.000000,0.400000,0.000000,0.200000,0.400000,2.750000,5
9,201201060028,199901120003,Klavon's Ice Cream Parlor,1999-01-05,Restaurant without Liquor,2012-01-06,0.0,0.0,0.0,4iSVxQKK1G3V-LBffQxaKg,...,0.000000,0.300000,0.600000,0.200000,0.100000,0.200000,0.200000,1.000000,4.000000,10


In [66]:
iy12th.to_csv('../dataset/iy12th2.csv')

In [65]:
# iy12th.to_csv('../dataset/iy12th.csv') # Old version (without category, attribute, hygenic dictionary words)